In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from pathlib import Path

In [ ]:
run_student_code = False

In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append(str(Path('../../')))

In [ ]:
if run_student_code:
    from exercises.camera_calibration.calibrated_lane_detector import CalibratedLaneDetector
else:
    from solutions.camera_calibration.calibrated_lane_detector import CalibratedLaneDetector

In [ ]:
# TODO: Change the next line(s), to create an instance of *your* LaneDetector
model_path = Path("../../solutions/lane_detection/best_model_multi_dice_loss.pth")
cld = CalibratedLaneDetector(model_path=model_path)

## Test on an image

Let us load an image

In [ ]:
# TODO: Change to another picture for which we know pitch and yaw. Maybe should have pitch -4, and yaw 2.5 
image_fn = str(Path("../../../data/Town04_Clear_Noon_09_09_2020_14_57_22_frame_625_validation_set.png"))
image = cv2.imread(image_fn)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.imshow(image[:, :, ::-1])
image.shape

First we detect the left and right boundaries as usual

In [ ]:
_, left_probs, right_probs = cld.detect(image)
# just to visualize both detections (left and right) in one image we add them up
plt.imshow(left_probs + right_probs, cmap="gray")

Next we fit straight lines to the left and right boundary

In [ ]:
line_left  = cld._fit_line_v_of_u(left_probs)
line_right = cld._fit_line_v_of_u(right_probs)

Let us visualize the straight lines

In [ ]:
def plot_detected_lines(line_left, line_right):
    u = np.arange(0,cld.cg.image_width, 1)
    v_left = line_left(u)
    v_right = line_right(u)

    plt.plot(u,v_left, color='r')
    plt.plot(u,v_right, color='b')
    plt.xlim(0,cld.cg.image_width)
    plt.ylim(cld.cg.image_height,0)

plt.imshow(left_probs + right_probs, cmap="gray")
plot_detected_lines(line_left, line_right)

Now compute the vanishing point

In [ ]:
vanishing_point = cld.get_intersection(line_left, line_right)
print(vanishing_point)

Visualize the vanishing point

In [ ]:
u_i, v_i = vanishing_point
plt.scatter([u_i],[v_i], marker="o", s=100, color="c", zorder=10)
plt.imshow(left_probs + right_probs, cmap="gray")
plot_detected_lines(line_left, line_right)

Finally determine pitch and yaw

In [ ]:
pitch, yaw = cld.get_py_from_vp(u_i, v_i, cld.cg.intrinsic_matrix)
print("pitch = ", np.rad2deg(pitch))
print("yaw = ",  np.rad2deg(yaw))

## Test on a video

The `calibrate` method of `CalibratedLaneDetector` does all these steps at once and maintains a queue of latest predictions. Each time a new prediction is added, the mean of all these observations are calculated and and used to update the `CameraGeometry` which is responsibe for converting lanes from image space to world space as we saw in earlier chapters. 

In [ ]:
video_filename = Path("../../../data/calibration_video.mp4")

cap = cv2.VideoCapture(str(video_filename))
while cap.isOpened():
    ret, image = cap.read()
    if not ret:
        break
    cld.calibrate(image[:,:,::-1])